In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import fitsio as F
from desitarget.targetmask import obsconditions, bgs_mask, desi_mask
from astropy.table import Table, join
import glob, os
from random import random
import math
import h5py

In [2]:
def select_cat_dat(cat):
    path="/global/cfs/cdirs/desi/datachallenge/onepercent/targets/"
    if cat=='BGS':
        targ=F.read(path+"mtl-bright.fits",columns=["TARGETID","RA","DEC","FLUX_G","FLUX_R","FLUX_Z","NOBS_G","NOBS_R","NOBS_Z","DESI_TARGET","BGS_TARGET","MWS_TARGET"])
        truth=F.read(path+"truth-bright.fits",ext=1,columns=["TARGETID","TRUEZ","TRUESPECTYPE","TEMPLATETYPE"])
        truth=truth[truth["TEMPLATETYPE"]=='BGS       ']
        sel = (targ["DESI_TARGET"] & desi_mask["BGS_ANY"]) > 0
        targ = targ[sel]
        #targ=targ[targ["DESI_TARGET"]==desi_mask.mask("BGS_ANY")]
    if cat=='LRG':
        targ=F.read(path+"mtl-dark.fits",columns=["TARGETID","RA","DEC","FLUX_G","FLUX_R","FLUX_Z","NOBS_G","NOBS_R","NOBS_Z","DESI_TARGET","BGS_TARGET","MWS_TARGET"])
        truth=F.read(path+"truth-dark.fits",ext=1,columns=["TARGETID","TRUEZ","TRUESPECTYPE","TEMPLATETYPE"])
        truth=truth[truth["TEMPLATETYPE"]=='LRG       ']
        sel = (targ["DESI_TARGET"] & desi_mask["LRG"]) > 0
        targ = targ[sel]
        #targ=targ[targ["DESI_TARGET"]==desi_mask.mask("LRG")]
    if cat=='QSO':
        targ=F.read(path+"mtl-dark.fits",columns=["TARGETID","RA","DEC","FLUX_G","FLUX_R","FLUX_Z","NOBS_G","NOBS_R","NOBS_Z","DESI_TARGET","BGS_TARGET","MWS_TARGET"])
        truth=F.read(path+"truth-dark.fits",ext=1,columns=["TARGETID","TRUEZ","TRUESPECTYPE","TEMPLATETYPE"])
        truth=truth[truth["TEMPLATETYPE"]=='QSO       ']
        sel = (targ["DESI_TARGET"] & desi_mask["QSO"]) > 0
        targ = targ[sel]
        #targ=targ[targ["DESI_TARGET"]==desi_mask.mask("QSO")]
    if cat=='ELG':
        targ=F.read(path+"mtl-dark.fits",columns=["TARGETID","RA","DEC","FLUX_G","FLUX_R","FLUX_Z","NOBS_G","NOBS_R","NOBS_Z","DESI_TARGET","BGS_TARGET","MWS_TARGET"])
        truth=F.read(path+"truth-dark.fits",ext=1,columns=["TARGETID","TRUEZ","TRUESPECTYPE","TEMPLATETYPE"])
        truth=truth[truth["TEMPLATETYPE"]=='ELG       ']
        sel = (targ["DESI_TARGET"] & desi_mask["ELG"]) > 0
        targ = targ[sel]
        #targ=targ[targ["DESI_TARGET"]==desi_mask.mask("ELG")]
    targ=Table(targ)
    truth=Table(truth)
    all_tar=join(targ,truth)
    # Keep cols: ["RA","DEC","TARGETID","TRUEZ"]
    #all_tar=all_tar["RA","DEC","TARGETID","TRUEZ"]
    all_tar.rename_column('TRUEZ', 'Z')
    rmag=np.where(all_tar["FLUX_R"]>0,22.-2.5*np.log10(all_tar["FLUX_R"]),0)
    all_tar["RMAG"]=rmag
    return all_tar

In [3]:
def get_bitweights(array):
    """
    Creates an array of bitwise weights stored as 64-bit signed integers
    Input: a 2D boolean array of shape (Ngal, Nreal), where Ngal is the total number 
           of target galaxies, and Nreal is the number of fibre assignment realizations.
    Output: returns a 2D array of 64-bit signed integers. 
    """
    Nbits=64
    dtype=np.int64
    Ngal, Nreal = array.shape           # total number of realizations and number of target galaxies
    Nout = (Nreal + Nbits - 1) // Nbits # number of output columns
    # intermediate arrays
    bitw8 = np.zeros((Ngal, 8), dtype="i")   # array of individual bits of 8 realizations
    bitweights = np.zeros(Ngal, dtype=dtype) # array of 64-bit integers
    # array to store final output
    output_array = np.zeros((Ngal, Nout), dtype=dtype)
    idx_out = 0 # initial column in output_array
    # loop through realizations to build bitwise weights
    for i in range(Nreal):
        bitw8[array[:,i], i%8] = 1
        arr = np.array(np.packbits(bitw8[:,::-1]), dtype=dtype)
        bitweights = np.bitwise_or(bitweights, np.left_shift(arr, 8*((i%Nbits)//8)))
        if (i+1)%Nbits == 0 or i+1 == Nreal:
            output_array[:,idx_out] = bitweights
            bitweights[:] = 0
            idx_out += 1
        if (i+1)%8 == 0:
            bitw8[:] = 0
    return output_array

In [4]:
def select_cat_ran(cat):
    path="/global/cfs/cdirs/desi/datachallenge/onepercent/catalogs/"
    columns=['TARGETID','RA','DEC','BRICKNAME','NOBS_G','NOBS_R','NOBS_Z','HPXPIXEL','DESI_TARGET','NUMOBS_INIT','PRIORITY','OBSCONDITIONS']
    if cat=='BGS':
        randoms=Table(F.read(path+"bright/randoms_mtl_cuttod.fits",columns=columns))
    if cat=='LRG':
        randoms=Table(F.read(path+"dark/randoms_mtl_cuttod.fits",columns=columns))
    if cat=='QSO':
        randoms=Table(F.read(path+"dark/randoms_mtl_cuttod.fits",columns=columns))    
    if cat=='ELG':
        randoms=Table(F.read(path+"gray/randoms_mtl_cuttod.fits",columns=columns))
    return randoms

In [5]:
#data=select_cat_dat('BGS')
#randoms=select_cat_ran('BGS')

In [6]:
def generate_pip_files(cat,realizations=2):
    data=select_cat_dat(cat)
    randoms=select_cat_ran(cat)
    if cat=='BGS':
        path='fiberassign_pip/bgs/fba_bright_targets_BGS'
    if cat=='LRG':
        path='fiberassign_pip/lrg/fba_dark_targets_LRG'
    if cat=='QSO':
        path='fiberassign_pip/qso/fba_dark_targets_QSO'
    if cat=='ELG':
        path='fiberassign_pip/elg/fba_dark_targets_ELG'
    for i in range(realizations):
        fba_files = glob.glob(path+str(i)+"/fba*.fits")
        allavail_tar=np.concatenate([F.read(f,ext="FAVAIL") for f in fba_files])
        allassign_tar=np.concatenate([F.read(f,ext="FASSIGN",columns=["FIBER","TARGETID","LOCATION","FIBERSTATUS"]) for f in fba_files])
        isas=np.zeros(len(data),dtype=bool)
        idas=np.isin(data["TARGETID"], allassign_tar["TARGETID"])
        isas[idas] = True
        data["ISASS_"+str(i)]=isas
    isav=np.zeros(len(data),dtype=bool)
    idav=np.isin(data["TARGETID"], allavail_tar["TARGETID"])
    isav[idav] = True
    data["ISAVAIL"]=isav
    
    parent=data[data["ISAVAIL"]==True]
    targeted=parent[parent["ISASS_0"]==True]
    col=["ISASS_"+str(i) for i in range(realizations)]
    targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
    parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))
    
    nd=len(parent)
    randoms["Z"]=np.zeros(len(randoms))

    for i in range(0,len(randoms)):
        ind=int(random()*nd)
        randoms["Z"][i]=parent["Z"][ind]

    a=get_bitweights(np.array([targeted["ISASS_"+str(i)] for i in range(realizations)]).T)

    for i in range(a.shape[1]):
        targeted["BITWEIGHT"+str(i)]=a[:,i]
        parent["BITWEIGHT"+str(i)]=np.ones(len(parent),dtype=int)*-1
        randoms["BITWEIGHT"+str(i)]=np.ones(len(randoms),dtype=int)*-1
    
    # columns to keep..
    base_cols1=["RA","DEC","Z","RMAG"]
    base_cols2=["RA","DEC","Z"]
    keep_cols=[col for col in targeted.colnames if 'BITWEIGHT' in col]
    all_cols1=base_cols1+keep_cols
    all_cols2=base_cols2+keep_cols
    parent0=parent[all_cols1]
    targeted0=targeted[all_cols1]
    randoms0=randoms[all_cols2]
    #targeted1=targeted0[targeted0["BITWEIGHT0"]%2==1]
    return parent0, targeted0, randoms0, targeted

In [7]:
BGS=generate_pip_files('BGS',128)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [8]:
LRG=generate_pip_files('LRG',128)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [9]:
ELG=generate_pip_files('ELG',128)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [12]:
BGS[2]

RA,DEC,Z,BITWEIGHT0,BITWEIGHT1
float64,float64,float64,int64,int64
6.21848473902398,7.284267317730249,0.25884509086608887,-1,-1
115.15158008450615,27.302728827147224,0.344650000333786,-1,-1
16.084427054953565,5.2305258313852345,0.23235926032066345,-1,-1
3.576222428472608,4.831808487476173,0.18943652510643005,-1,-1
119.7874436440232,28.52975579427368,0.17299695312976837,-1,-1
15.491440984871481,0.3093129124309112,0.14450670778751373,-1,-1
113.49696483190469,35.8370534896702,0.3511914014816284,-1,-1
2.2973137600455686,0.09410391454029275,0.2984442710876465,-1,-1
118.75721413033553,35.06152454012639,0.14659903943538666,-1,-1


In [13]:
def writefiles(parent,targeted,randoms,cat,realizations=128,rmag=30):
    rmag_cut1=targeted["RMAG"]<=rmag
    targeted=targeted[rmag_cut1]
    rmag_cut2=parent["RMAG"]<=rmag
    parent=parent[rmag_cut2]
    cols_bitweight=[col for col in targeted.colnames if 'BITWEIGHT' in col]
    outdir="clus_files_pip2/"
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    with h5py.File(outdir+'parent_'+cat+'.hdf5', 'w') as f:
        f.create_dataset("RA", data=parent["RA"])
        f.create_dataset("DEC", data=parent["DEC"])
        f.create_dataset("Z", data=parent["Z"])
        for i in range(math.ceil(realizations/64)):
            f.create_dataset("BITWEIGHT"+str(i), data=parent["BITWEIGHT"+str(i)])
    with h5py.File(outdir+'randoms_'+cat+'.hdf5', 'w') as f:
        f.create_dataset("RA", data=randoms["RA"])
        f.create_dataset("DEC", data=randoms["DEC"])
        f.create_dataset("Z", data=randoms["Z"])
        for i in range(math.ceil(realizations/64)):
            f.create_dataset("BITWEIGHT"+str(i), data=randoms["BITWEIGHT"+str(i)])
    with h5py.File(outdir+'targeted_'+cat+'.hdf5', 'w') as f:
        f.create_dataset("RA", data=targeted["RA"])
        f.create_dataset("DEC", data=targeted["DEC"])
        f.create_dataset("Z", data=targeted["Z"])
        for i in range(math.ceil(realizations/64)):
            f.create_dataset("BITWEIGHT"+str(i), data=targeted["BITWEIGHT"+str(i)])

def writefiles_iip(targeted,randoms,cat,realizations=128,rmag=25):
    rmag_cut=targeted["RMAG"]<=rmag
    targeted=targeted[rmag_cut]
    outdir="clus_files_pip2/"
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    with h5py.File(outdir+'targeted_iip_'+cat+'.hdf5', 'w') as f:
        f.create_dataset("RA", data=targeted["RA"])
        f.create_dataset("DEC", data=targeted["DEC"])
        f.create_dataset("Z", data=targeted["Z"])
        f.create_dataset("weight", data=targeted["wi"]/realizations)
    with h5py.File(outdir+'randoms_iip_'+cat+'.hdf5', 'w') as f:
        f.create_dataset("RA", data=randoms["RA"])
        f.create_dataset("DEC", data=randoms["DEC"])
        f.create_dataset("Z", data=randoms["Z"])
        f.create_dataset("weight", data=np.ones(len(randoms)))

In [14]:
writefiles(BGS[0],BGS[1],BGS[2],'bgs128_rmag19.0',128,19.0)
writefiles_iip(BGS[3],BGS[2],'bgs128_rmag19.0',128,19.0)

In [12]:
#writefiles(LRG[0],LRG[1],LRG[2],'lrg128_rmag25.0',128,25.0)
#writefiles_iip(LRG[3],LRG[2],'lrg128_rmag25.0',128,25.0)

In [13]:
#writefiles(ELG[0],ELG[1],ELG[2],'elg128_rmag25.0',128,25.0)
#writefiles_iip(ELG[3],ELG[2],'elg128_rmag25.0',128,25.0)

In [14]:
#print(np.unpackbits(np.array([n1], dtype=np.uint64).view(np.uint8)), np.count_nonzero(np.unpackbits(np.array([n1], dtype=np.uint64).view(np.uint8))))
#print(np.unpackbits(np.array([n2], dtype=np.uint64).view(np.uint8)), np.count_nonzero(np.unpackbits(np.array([n2], dtype=np.uint64).view(np.uint8))))

In [15]:
LRG112=generate_pip_files('LRG',112)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [16]:
LRG96=generate_pip_files('LRG',96)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [17]:
LRG80=generate_pip_files('LRG',80)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [18]:
LRG64=generate_pip_files('LRG',64)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [19]:
LRG48=generate_pip_files('LRG',48)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [20]:
LRG32=generate_pip_files('LRG',32)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [21]:
LRG16=generate_pip_files('LRG',16)

<ipython-input-6-f1ac99990736>:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  targeted["wi"]=np.sum(targeted["ISASS_"+str(i)] for i in range(realizations))
<ipython-input-6-f1ac99990736>:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  parent["wi"]=np.sum(parent["ISASS_"+str(i)] for i in range(realizations))


In [22]:
writefiles(LRG112[0],LRG112[1],LRG112[2],'lrg112_rmag25.0',112,25.0)
writefiles(LRG96[0],LRG96[1],LRG96[2],'lrg96_rmag25.0',96,25.0)
writefiles(LRG80[0],LRG80[1],LRG80[2],'lrg80_rmag25.0',80,25.0)
writefiles(LRG64[0],LRG64[1],LRG64[2],'lrg64_rmag25.0',64,25.0)
writefiles(LRG48[0],LRG48[1],LRG48[2],'lrg48_rmag25.0',48,25.0)
writefiles(LRG32[0],LRG32[1],LRG32[2],'lrg32_rmag25.0',32,25.0)
writefiles(LRG16[0],LRG16[1],LRG16[2],'lrg16_rmag25.0',16,25.0)

In [11]:
def generate_pip_files_ran(cat,realizations=2):
    randoms=select_cat_ran(cat)
    data=select_cat_dat(cat)

    if cat=='BGS':
        path='fiberassign_pip/bgs_ran/fba_bright_randoms_BGS'
    if cat=='LRG':
        path='fiberassign_pip/lrg_ran/fba_bright_randoms_LRG'
    if cat=='ELG':
        path='fiberassign_pip/elg_ran/fba_bright_randoms_ELG'
    for i in range(realizations):
        fba_files = glob.glob(path+str(i)+"/fba*.fits")
        allavail_ran=np.concatenate([F.read(f,ext="FAVAIL") for f in fba_files])
        allassign_ran=np.concatenate([F.read(f,ext="FASSIGN",columns=["FIBER","TARGETID","LOCATION","FIBERSTATUS"]) for f in fba_files])
        isas=np.zeros(len(randoms),dtype=bool)
        idas=np.isin(randoms["TARGETID"], allassign_ran["TARGETID"])
        isas[idas] = True
        randoms["ISASS_"+str(i)]=isas
    isav=np.zeros(len(randoms),dtype=bool)
    idav=np.isin(randoms["TARGETID"], allavail_ran["TARGETID"])
    isav[idav] = True
    randoms["ISAVAIL"]=isav
    
    col=["ISASS_"+str(i) for i in range(realizations)]
    randoms["wi"]=np.sum(randoms["ISASS_"+str(i)] for i in range(realizations))
    
    nd=len(data)
    randoms["Z"]=np.zeros(len(randoms))

    for i in range(0,len(randoms)):
        ind=int(random()*nd)
        randoms["Z"][i]=data["Z"][ind]

    a=get_bitweights(np.array([randoms["ISASS_"+str(i)] for i in range(realizations)]).T)

    for i in range(a.shape[1]):
        randoms["BITWEIGHT"+str(i)]=a[:,i]
        
    # columns to keep..
    base_cols2=["RA","DEC","Z","wi"]
    keep_cols=[col for col in randoms.colnames if 'BITWEIGHT' in col]
    all_cols2=base_cols2+keep_cols
    randoms0=randoms[all_cols2]
    #targeted1=targeted0[targeted0["BITWEIGHT0"]%2==1]
    return randoms0

In [12]:
random_BGSiip=generate_pip_files_ran('BGS',realizations=128)

<ipython-input-11-bfaa8a512015>:25: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  randoms["wi"]=np.sum(randoms["ISASS_"+str(i)] for i in range(realizations))


In [17]:
random_ELGiip=generate_pip_files_ran('ELG',realizations=128)

<ipython-input-11-bfaa8a512015>:25: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  randoms["wi"]=np.sum(randoms["ISASS_"+str(i)] for i in range(realizations))


In [18]:
random_LRGiip=generate_pip_files_ran('LRG',realizations=128)

<ipython-input-11-bfaa8a512015>:25: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  randoms["wi"]=np.sum(randoms["ISASS_"+str(i)] for i in range(realizations))


In [19]:
random_BGSiip["wi"].max(), random_LRGiip["wi"].max(), random_ELGiip["wi"].max()

(119, 128, 121)

In [21]:
def writefiles_iip_ran(randoms,cat,realizations=128):
    outdir="clus_files_pip2/"
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    with h5py.File(outdir+'randoms_iip2_'+cat+'.hdf5', 'w') as f:
        f.create_dataset("RA", data=randoms["RA"])
        f.create_dataset("DEC", data=randoms["DEC"])
        f.create_dataset("Z", data=randoms["Z"])
        f.create_dataset("weight", data=randoms["wi"]/realizations)

In [25]:
writefiles_iip_ran(random_BGSiip,'bgs_rmag19.0',128)
writefiles_iip_ran(random_LRGiip,'lrg_rmag25.0',128)
writefiles_iip_ran(random_ELGiip,'elg_rmag25.0',128)